In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import ast
import json

In [57]:
# papers = list(range(20))
# leads = list(range(5))
# editors = list(range(15))
#
# papers_per_team = 5
#
# c_ik = np.random.rand(len(editors), len(papers))
# d = np.random.rand(len(leads), len(papers))
# c = np.array([c_ik for _ in leads])

In [21]:
editors_df = pd.read_csv("areas_of_interest_old.csv").iloc[:, 1:]
int_map = dict(zip(editors_df['Name'], editors_df['Areas of Interest']))

abstracts_inc = pd.read_csv("abstracts_inc.csv")
paper_map = dict(zip(abstracts_inc['ID'], abstracts_inc['Abstract']))

with open('rewards.json', 'r') as file:
    c_str_2 = json.load(file)
c_2 = {ast.literal_eval(key): ast.literal_eval(value) for key, value in c_str_2.items()}

In [31]:
editors = list(editors_df['Name'].iloc[3:])
leads = list(editors_df['Name'].iloc[:3])
papers = list(paper_map.keys())

In [68]:
m = gp.Model()

x = m.addVars(editors, leads, vtype=GRB.BINARY, name="editor_lead")
y = m.addVars(leads, papers, vtype=GRB.BINARY, name="lead_paper")
z = m.addVars(editors, leads, papers, vtype=GRB.BINARY, name="paper_lead_editor")

m.update()

In [69]:
# EACH EDITOR ONLY ASSIGNED TO ONE LEAD / TEAM
editor_one_lead = m.addConstrs(gp.quicksum(x[i, j] for j in leads) == 1 for i in editors)

# EACH EDITOR GETS SIMILAR # OF EDITORS
lead_team = m.addConstrs(gp.quicksum(x[i, j] for i in editors) >= 2 for j in leads)

# EACH PAPER ASSIGNED TO AT LEAST ONE TEAM
paper_one_team = m.addConstrs(gp.quicksum(y[j, k] for j in leads) >= 1 for k in papers)

# EACH TEAM GETS SPECIFIED # OF PAPERS (PARAMETER)
paper_count = m.addConstrs(gp.quicksum(y[j, k] for k in papers) == 4 for j in leads)

# RELATE VARIABLES
relate = m.addConstrs(z[i, j, k] == x[i, j] * y[j, k] for i in editors for j in leads for k in papers)

m.update()

In [70]:
m.setObjective(gp.quicksum(c_2[k, i] * z[i, j, k] for i in editors for j in leads for k in papers) + gp.quicksum(c_2[k, j] * y[j, k] for j in leads for k in papers))
m.update()

In [71]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 23 rows, 249 columns and 102 nonzeros
Model fingerprint: 0x81dbc056
Model has 198 quadratic constraints
Variable types: 0 continuous, 249 integer (249 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [5e-02, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 815 rows, 447 columns, 1884 nonzeros
Variable types: 0 continuous, 447 integer (447 binary)
Found heuristic solution: objective 10.5560523
Found heuristic solution: objective 9.6130633

Root relaxation: objective 2.989840e+00, 96 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |    

In [72]:
for v in m.getVars():
    if v.VType == GRB.BINARY and v.X > 0.5:
        print(f"{v.VarName}: {v.X}")

editor_lead[Matthew Ahn,Bryson Jun]: 1.0
editor_lead[Muriel Taylor-Adair,Kelly Guo]: 1.0
editor_lead[Rijuta Vallishayee,Bryson Jun]: 1.0
editor_lead[Sophia Cho,Karina Pan]: 1.0
editor_lead[Sophia Govea,Karina Pan]: 1.0
editor_lead[Gina Matos,Kelly Guo]: 1.0
lead_paper[Bryson Jun,3_Realities]: 1.0
lead_paper[Bryson Jun,5_Resilience]: 1.0
lead_paper[Bryson Jun,7_Friend]: 1.0
lead_paper[Bryson Jun,10_KimIlSung]: 1.0
lead_paper[Karina Pan,4_Conditions]: 1.0
lead_paper[Karina Pan,8_GenderGap]: 1.0
lead_paper[Karina Pan,13_Singapore]: 1.0
lead_paper[Karina Pan,14_Persecution]: 1.0
lead_paper[Kelly Guo,2_Cholera]: 1.0
lead_paper[Kelly Guo,5_Resilience]: 1.0
lead_paper[Kelly Guo,6_Biointelligence]: 1.0
lead_paper[Kelly Guo,9_Movement]: 1.0
paper_lead_editor[Matthew Ahn,Bryson Jun,3_Realities]: 1.0
paper_lead_editor[Matthew Ahn,Bryson Jun,5_Resilience]: 1.0
paper_lead_editor[Matthew Ahn,Bryson Jun,7_Friend]: 1.0
paper_lead_editor[Matthew Ahn,Bryson Jun,10_KimIlSung]: 1.0
paper_lead_editor[Murie

gurobipy._core.tupledict